In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17067, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 17067 (delta 24), reused 27 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17067/17067), 15.69 MiB | 16.16 MiB/s, done.
Resolving deltas: 100% (11718/11718), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.1/899.1 kB 31.4 MB/s eta 0:00:00


In [ ]:
import torch
import utils
display = utils.notebook_init()

YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.7/112.6 GB disk)


In [ ]:
!nvidia-smi

Wed Nov 27 05:37:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              11W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import os
import shutil

# Define source and destination directories
source_dir = '/content/drive/MyDrive/Colab_Notebooks/val2017'
destination_dir = '/content/dataset/images/train'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# List all files in the source directory
all_images = os.listdir(source_dir)

# Ensure the number of files in the source directory is sufficient
if len(all_images) < 1000:
    print("Not enough images in the source directory.")
else:
    # Select the first 100 images
    selected_images = all_images[:1000]

    # Copy selected images to the destination directory
    for image in selected_images:
        shutil.copy(os.path.join(source_dir, image), os.path.join(destination_dir, image))

    print(f"Copied {len(selected_images)} images to {destination_dir}")

Copied 1000 images to /content/dataset/images/train


In [4]:
import os
import json
from PIL import Image

# Define paths
images_dir = '/content/dataset/images/train'  # Directory with 100 images
annotations_file = '/content/drive/MyDrive/Colab_Notebooks/instances_val2017.json/instances_val2017.json'  # COCO annotations file
output_dir = '/content/dataset/labels/train'  # Directory to save YOLO annotations

# Create output directory if not exists
os.makedirs(output_dir, exist_ok=True)

# Load COCO annotations
with open(annotations_file, 'r') as f:
    coco_data = json.load(f)

# Build a mapping from image IDs to file names
image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}

# Build a mapping from image IDs to their annotations
image_id_to_annotations = {}
for ann in coco_data["annotations"]:
    img_id = ann["image_id"]
    bbox = ann["bbox"]  # [x, y, width, height]
    category_id = ann["category_id"]
    if img_id not in image_id_to_annotations:
        image_id_to_annotations[img_id] = []
    image_id_to_annotations[img_id].append({"bbox": bbox, "category_id": category_id})

# Process only the 100 selected images
for image_file in os.listdir(images_dir):
    image_path = os.path.join(images_dir, image_file)
    if image_file.endswith(('.jpg', '.png')):  # Only process image files
        # Get the image ID from its filename
        img_id = next((id for id, file in image_id_to_filename.items() if file == image_file), None)
        if img_id is None:
            print(f"No annotations found for {image_file}")
            continue

        # Open the image to get its dimensions
        with Image.open(image_path) as img:
            image_width, image_height = img.size

        # Get annotations for the image
        annotations = image_id_to_annotations.get(img_id, [])

        # Create a YOLO-format annotation file
        yolo_annotation_path = os.path.join(output_dir, os.path.splitext(image_file)[0] + '.txt')
        with open(yolo_annotation_path, 'w') as f:
            for ann in annotations:
                # Normalize bounding box
                x, y, width, height = ann["bbox"]
                x_center = (x + width / 2) / image_width
                y_center = (y + height / 2) / image_height
                norm_width = width / image_width
                norm_height = height / image_height

                # Write in YOLO format
                f.write(f"{ann['category_id']} {x_center} {y_center} {norm_width} {norm_height}\n")

        print(f"Processed {image_file}")

print("YOLO conversion complete.")


Processed 000000572555.jpg
Processed 000000474170.jpg
Processed 000000491470.jpg
Processed 000000560474.jpg
Processed 000000476787.jpg
Processed 000000570834.jpg
Processed 000000472030.jpg
Processed 000000530457.jpg
Processed 000000493772.jpg
Processed 000000462904.jpg
Processed 000000537506.jpg
Processed 000000537153.jpg
Processed 000000476810.jpg
Processed 000000474344.jpg
Processed 000000569700.jpg
Processed 000000530099.jpg
Processed 000000509824.jpg
Processed 000000555705.jpg
Processed 000000550714.jpg
Processed 000000485480.jpg
Processed 000000546829.jpg
Processed 000000503823.jpg
Processed 000000484351.jpg
Processed 000000539962.jpg
Processed 000000489014.jpg
Processed 000000497568.jpg
Processed 000000492362.jpg
Processed 000000557672.jpg
Processed 000000480122.jpg
Processed 000000468124.jpg
Processed 000000491497.jpg
Processed 000000507975.jpg
Processed 000000522713.jpg
Processed 000000489842.jpg
Processed 000000545219.jpg
Processed 000000558213.jpg
Processed 000000577735.jpg
P

In [5]:
# Create a train.txt file listing all image paths
with open('train.txt', 'w') as f:
    for image_file in os.listdir(images_dir):
        if image_file.endswith(('.jpg', '.png')):
            f.write(f"{os.path.abspath(os.path.join(images_dir, image_file))}\n")

In [7]:
import os

# Path to labels directory
labels_dir = '/content/dataset/labels/train'
valid_classes = range(90)  # Adjust based on nc value in data.yaml

for label_file in os.listdir(labels_dir):
    file_path = os.path.join(labels_dir, label_file)
    with open(file_path, 'r') as f:
        for line in f.readlines():
            class_id = int(line.split()[0])
            if class_id not in valid_classes:
                print(f"Invalid label {class_id} in file {label_file}")


Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000560474.txt
Invalid label 90 in file 000000500464.txt
Invalid label 90 in file 000000500464.txt
Invalid label 90 in file 000000521405.txt
Invalid label 90 in file 000000500565.txt
Invalid label 90 in file 000000482319.txt
Invalid label 90 in file 000000482319.txt
Invalid label 90 in file 000000482319.txt
Invalid label 90 in file 000000544519.txt
Invalid label 90 in file 000000465179.txt
Invalid label 90 in file 000000465179.txt
Invalid label 90 in file 000000578922.txt
Invalid label 90 in file 000000569917.txt
Invalid label 90 in file 000000569917.txt
Invalid label 90 in file 000000563281.txt
Invalid label 90 in file 000000580

In [8]:
import os

# Paths to your dataset
images_path = '/content/dataset/images/train'
labels_path = '/content/dataset/labels/train'

# Invalid label to check
invalid_label = '90'

# Find and delete files with invalid labels
for label_file in os.listdir(labels_path):
    if label_file.endswith('.txt'):
        label_file_path = os.path.join(labels_path, label_file)

        # Check for invalid label in the annotation file
        with open(label_file_path, 'r') as file:
            lines = file.readlines()

        if any(line.startswith(invalid_label) for line in lines):
            # Delete the annotation file
            os.remove(label_file_path)

            # Construct the corresponding image file path
            image_file_name = os.path.splitext(label_file)[0] + '.jpg'
            image_file_path = os.path.join(images_path, image_file_name)

            # Delete the image file if it exists
            if os.path.exists(image_file_path):
                os.remove(image_file_path)
                print(f"Deleted {image_file_name} and {label_file}")
            else:
                print(f"Deleted {label_file}, but image file not found.")


Deleted 000000560474.jpg and 000000560474.txt
Deleted 000000500464.jpg and 000000500464.txt
Deleted 000000521405.jpg and 000000521405.txt
Deleted 000000500565.jpg and 000000500565.txt
Deleted 000000482319.jpg and 000000482319.txt
Deleted 000000544519.jpg and 000000544519.txt
Deleted 000000465179.jpg and 000000465179.txt
Deleted 000000578922.jpg and 000000578922.txt
Deleted 000000569917.jpg and 000000569917.txt
Deleted 000000563281.jpg and 000000563281.txt
Deleted 000000580294.jpg and 000000580294.txt
Deleted 000000507473.jpg and 000000507473.txt
Deleted 000000492878.jpg and 000000492878.txt


In [9]:
import os

labels_dir = '/content/dataset/labels/train'
valid_classes = range(90)  # Adjust based on your nc value

for label_file in os.listdir(labels_dir):
    file_path = os.path.join(labels_dir, label_file)
    with open(file_path, 'r') as f:
        for line in f.readlines():
            class_id = int(line.split()[0])
            if class_id not in valid_classes:
                print(f"Invalid label {class_id} in file {label_file}")

In [11]:
import os

labels_path = '/content/dataset/labels/train'  # Update path if necessary
max_class = 79  # Maximum valid class index
errors = []

for label_file in os.listdir(labels_path):
    if label_file.endswith('.txt'):
        label_file_path = os.path.join(labels_path, label_file)
        with open(label_file_path, 'r') as file:
            lines = file.readlines()

        valid_lines = []
        for line in lines:
            class_index = int(line.split()[0])
            if class_index > max_class:
                errors.append((label_file, class_index))
            else:
                valid_lines.append(line)

        # Overwrite file with valid labels only
        with open(label_file_path, 'w') as file:
            file.writelines(valid_lines)

if errors:
    print("Invalid labels detected and removed:")
    for file, cls in errors:
        print(f"File: {file}, Invalid Class: {cls}")
else:
    print("All label files are valid!")

Invalid labels detected and removed:
File: 000000565469.txt, Invalid Class: 85
File: 000000549220.txt, Invalid Class: 84
File: 000000519569.txt, Invalid Class: 86
File: 000000466125.txt, Invalid Class: 84
File: 000000572678.txt, Invalid Class: 86
File: 000000507081.txt, Invalid Class: 82
File: 000000577735.txt, Invalid Class: 86
File: 000000505789.txt, Invalid Class: 85
File: 000000472678.txt, Invalid Class: 84
File: 000000472678.txt, Invalid Class: 84
File: 000000546826.txt, Invalid Class: 87
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 84
File: 000000520077.txt, Invalid Class: 

In [12]:
!python train.py --img 640 --batch 16 --epochs 10 --data /content/dataset/data.yaml --weights yolov5s.pt --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-05 05:06:22.034476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-05 05:06:22.056168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-05 05:06:22.067593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

In [13]:
!python val.py --weights runs/train/exp2/weights/best.pt --data /content/dataset/data.yaml --img-size 640 --batch-size 16

val: data=/content/dataset/data.yaml, weights=['runs/train/exp2/weights/best.pt'], batch_size=16, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning /content/dataset/labels/train.cache... 987 images, 45 backgrounds, 0 corrupt: 100% 987/987 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 62/62 [00:17<00:00,  3.47it/s]
                   all        987       6696      0.768      0.655      0.731      0.475
Speed: 0.2ms pre-process, 5.1ms inference, 4.2ms NMS per image at shape (16, 3, 640, 640)
Results saved to runs/val

In [26]:
!python detect.py --weights runs/train/exp2/weights/best.pt --img 640 --source /content/dataset/images/train --conf-thres 0.1 --save-txt --save-conf

detect: weights=['runs/train/exp2/weights/best.pt'], source=/content/dataset/images/train, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.1, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-388-g882c35fc Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/987 /content/dataset/images/train/000000462031.jpg: 640x480 2 bicycles, 1 surfboard, 1 bottle, 1 wine glass, 28.6ms
image 2/987 /content/dataset/images/train/000000462371.jpg: 544x640 7 bicycles, 1 tie, 7 sports balls, 2 tennis rackets, 1 apple, 1 toilet, 

In [32]:
import os

labels_path = "runs/detect/exp8/labels"  # Path to the labels folder
confidence_threshold = 0.8  # 80% threshold

count = 0
if not os.path.exists(labels_path):
    print(f"Labels directory '{labels_path}' does not exist.")
else:
    for file_name in os.listdir(labels_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(labels_path, file_name)
            with open(file_path, "r") as f:
                lines = f.readlines()
                for line in lines:
                    confidence = float(line.split()[5])  # Confidence is the 6th element
                    if confidence > confidence_threshold:
                        count += 1
                        break  # If one detection meets the threshold, count the image and stop checking further

    print(f"Number of images with confidence > {confidence_threshold * 100}%: {count}")

Number of images with confidence > 80.0%: 640


In [33]:
import os

labels_path = "runs/detect/exp8/labels"  # Path to the labels folder
confidence_threshold = 0.9  # 90% threshold

count = 0
if not os.path.exists(labels_path):
    print(f"Labels directory '{labels_path}' does not exist.")
else:
    for file_name in os.listdir(labels_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(labels_path, file_name)
            with open(file_path, "r") as f:
                lines = f.readlines()
                for line in lines:
                    confidence = float(line.split()[5])  # Confidence is the 6th element
                    if confidence > confidence_threshold:
                        count += 1
                        break  # If one detection meets the threshold, count the image and stop checking further

    print(f"Number of images with confidence > {confidence_threshold * 100}%: {count}")

Number of images with confidence > 90.0%: 189
